In [1]:
%cd /home/yokoyama/research
from types import SimpleNamespace
import sys
import os
from glob import glob

import cv2
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.manifold import TSNE
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm

sys.path.append(".")
from modules.individual import IndividualActivityRecognition, IndividualDataFormat, IndividualDataHandler
from modules.utils.constants import Stages
from modules.utils.video import Capture, Writer

/raid6/home/yokoyama/research


In [2]:
# set constants
model_type = "role_estimation"
data_type = "bbox"
seq_len = 150

train_data_dir = os.path.join("data", "dataset01", "train")
test_data_dir = os.path.join("data", "dataset01", "test")

In [3]:
# load results
def load_results(data_dir, model_type, data_type, seq_len, data_keys):
    results = {}
    # for data_dir in tqdm(sorted(glob(os.path.join(data_dir, "*")))[:1]):
    for data_dir in tqdm(sorted(glob(os.path.join(data_dir, "*")))):
        video_num = os.path.basename(data_dir)
        results[video_num] = IndividualDataHandler.load(data_dir, model_type, data_type, False, seq_len, data_keys)

    return results

In [4]:
data_keys = [IndividualDataFormat.bbox_real, IndividualDataFormat.role_label, IndividualDataFormat.role_aux]
train_results = load_results(train_data_dir, model_type, data_type, seq_len, data_keys)
# test_results = load_results(test_data_dir, model_type, data_type, seq_len, data_keys)

  0%|          | 0/20 [00:00<?, ?it/s]

In [5]:
from modules.visualize.pose import draw_bbox, put_frame_num


def write_frame(frame, ind_data_lst, frame_num):
    # add keypoints to image
    frame = put_frame_num(frame, frame_num)
    for data in ind_data_lst:
        if data[IndividualDataFormat.frame_num] == frame_num:
            idx = np.argmax(data[IndividualDataFormat.role_aux])
            if idx == 1:
                frame = draw_bbox(frame, np.array(data[IndividualDataFormat.bbox_real])[-1] * frame.shape[1::-1])

    return frame

In [ ]:
results = train_results
s = "train"

for video_num, result in results.items():
    print(video_num)
    data_dir = os.path.join("data", "dataset01", s, video_num)
    
    # load video
    video_path = os.path.join(data_dir, f"{video_num}_pose.mp4")
    cap = Capture(video_path)
    
    # save_video
    tmp_frame = cap.read()[1]
    cap.set_pos_frame_count(0)
    out_path = os.path.join(data_dir, f"{video_num}_role_{data_type}_seq{seq_len}.mp4")
    wtr = Writer(out_path, cap.fps, tmp_frame.shape[1::-1])
    
    # write frame
    for frame_num in tqdm(range(cap.frame_count)):
        frame_num += 1
        ret, frame = cap.read()
        frame = write_frame(frame, result, frame_num)
        wtr.write(frame)
        
del cap, wtr

01


  0%|          | 0/5400 [00:00<?, ?it/s]

02


  0%|          | 0/5400 [00:00<?, ?it/s]

03


  0%|          | 0/5400 [00:00<?, ?it/s]